# RHESSys model setup script

Model organization is documented on https://github.com/laurencelin/GIS2RHESSys.

***
## 1) RHESSys project directory & upload files

reload boxes 1-4 if thie script is restarted.

### 1.A define directory names

<div class="alert alert-block alert-info">
This block sets up RHESSys project names, GIS projection, and GIS spatial resolution. <b>User inputs are required.</b>
</div>

In [1]:
# -------------------------- project and RHESSys 
PROJDIR='jupyter_SWAS' 
RHESSysMODEL='PAIN'
# -------------------------- GIS spatial resolution and projection (UTM)
# look up from http://spatialreference.org/ref/epsg/?page=1
# EPSG:26917 = NAD83 UTM 17N
# EPSG:26918 = NAD83 UTM 18N
EPSGCODE='EPSG:26917' # NAD83 UTM 17N ***
RESOLUTION=30 #***
MAPSET='PERMANENT'

<div class="alert alert-block alert-info">
Two blocks below set the paths for the calculations in this script. Please DO NOT edit them.
</div>

In [2]:
%%bash --out USER
printf "$USER"

In [3]:
SCRATCH = '/scratch/'+ USER

In [5]:
RAWGISDIR=SCRATCH+'/'+PROJDIR+'/raw_data'
RHESSysDIR=SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL
GISDBASE=SCRATCH+'/'+PROJDIR+'/grass_dataset'
RBASE=SCRATCH+'/'+PROJDIR+'/RLIB'
LOCATION=GISDBASE+'/'+RHESSysMODEL
LOCATIONDEM=GISDBASE+'/'+'elevationRAW'
LOCATIONSOIL=GISDBASE+'/'+'soilRAW'
LOCATIONLULC=GISDBASE+'/'+'lulcRAW'

<div class="alert alert-block alert-success">
<b>Optional:</b> create directories if directories are not previously setup. It does not overwrite.
</div>

In [50]:
!mkdir {SCRATCH}/{PROJDIR}
!mkdir {RAWGISDIR}
!mkdir {RHESSysDIR}
!mkdir {GISDBASE}
!mkdir {RHESSysDIR}/flows
!mkdir {RHESSysDIR}/worldfiles
!mkdir {RHESSysDIR}/defs
!mkdir {RHESSysDIR}/tecfiles
!mkdir {RHESSysDIR}/code
!mkdir {RBASE}

mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/raw_data’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/PAIN’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/grass_dataset’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/PAIN/flows’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/PAIN/worldfiles’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/PAIN/defs’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/PAIN/tecfiles’: File exists
mkdir: cannot create directory ‘/scratch/hl8vq/jupyter_SWAS/RLIB’: File exists


<div class="alert alert-block alert-success">
<b>Optional:</b>  create GRASS database for the project if it is not previously setup. It does not overwrite.
</div>

In [7]:
!grass74 -c {EPSGCODE} -e {LOCATION}

Creating new GRASS GIS location/mapset...
Cleaning up temporary files...


<div class="alert alert-block alert-success">
<b>Optional:</b> By default, R packages are already installed on Rivanna. If not, please install R packages to work with GRASS using the script below.
</div>

In [ ]:
#-----------------------  make directory to hold scource codes from CRAN and download them
!wget -O {RBASE}/sp_1.3-1.tar.gz https://cran.r-project.org/src/contrib/sp_1.3-1.tar.gz
!wget -O {RBASE}/XML_3.98-1.16.tar.gz https://cran.r-project.org/src/contrib/XML_3.98-1.16.tar.gz
!wget -O {RBASE}/rgdal_1.3-6.tar.gz https://cran.r-project.org/src/contrib/rgdal_1.3-6.tar.gz
!wget -O {RBASE}/rgrass7_0.1-12.tar.gz https://cran.r-project.org/src/contrib/rgrass7_0.1-12.tar.gz
#----------------------- install the downloaded packages into R 3.5.x
!R CMD INSTALL {RBASE}/sp_1.3-1.tar.gz
!R CMD INSTALL {RBASE}/XML_3.98-1.16.tar.gz
!R CMD INSTALL {RBASE}/rgdal_1.3-6.tar.gz
!grass74 {LOCATION}/{MAPSET} --exec R CMD INSTALL {RBASE}/rgrass7_0.1-12.tar.gz

### 1.B upload data

<div class="alert alert-block alert-success">
<b>Option 1:</b> Upload files to Rivanna by yourself 
</div>

<div class="alert alert-block alert-info">
Upload files to the <project/raw_data>. You may use linux command for upload/download files on UVA Rivanna, which is the host cluster for this Jupyter Notebook. For example, scp -r <folder/files> [USER]@rivanna.hpc.virginia.edu:/scratch/[USER]/[Project]/raw_data. The command below shows what is in the project raw_data directory 
</div>


In [ ]:
ls -l {RAWGISDIR}

<div class="alert alert-block alert-info">
Then, set file names below. <b>User edits are required.</b>
</div>

In [ ]:
downloadedDEMfile = RAWGISDIR + '/' + 'swas_dem30.tif'
downloadedLULCfile = RAWGISDIR + '/' + 'NLCD.tif'
downloadedSSURGOdirectory = RAWGISDIR + '/' + 'MC005'
gageShapefile = RAWGISDIR + '/' + 'swas_gages_utm.shp' # (optional)

<div class="alert alert-block alert-success">
<b>Option 2:</b> Download GIS information from HydroShare (we use this option in this example)
</div>

For example,
<div class="alert alert-block alert-info">
Lin, L. (2019). SWAS raw GIS, HydroShare, http://www.hydroshare.org/resource/f52da10251c14c24bebfccbb32ccd614
</div>

Then, <b>copy the weblink below. </b> The following blocks help user to download HydroShare content to the project directory and unzip it. <b>User needs to identify the corresponding files for elevation, LULC, ...etc. in the last block where the red texts </b>

In [8]:
HydroShareLink = 'https://www.hydroshare.org/resource/f52da10251c14c24bebfccbb32ccd614/'
HydroShareID = HydroShareLink.split('/')[4]
HydroShareDownloadZip = HydroShareID +'.zip'

In [9]:
from hs_restclient import HydroShare
hs = HydroShare()
try:
    hs.getResource(HydroShareID, destination=RAWGISDIR, unzip=True)
except HydroShareBagNotReadyException as e:
    print('BagIt file is being generated and not ready for download at this time.')

Username: LaurenceJnote
Password for LaurenceJnote: ········


<div class="alert alert-block alert-info">
Unzipping the downloaded resources. We expect that the HydroShare resource is a zip package of GIS files and data
</div>

In [10]:
##--- related to --- hs.getResource(HydroShareID, destination=RAWGISDIR, unzip=True)
from os import listdir
import zipfile
unzippedpath = RAWGISDIR+'/'+HydroShareID+'/'+HydroShareID+"/data/contents/"
onlyfiles = [f for f in listdir(unzippedpath) if f.endswith(".zip")]
with zipfile.ZipFile((unzippedpath+onlyfiles[0]), 'r') as zip_ref:
    zip_ref.extractall(unzippedpath)

In [11]:
##--------------------------- listing unzip content ---------------------------##
from os import listdir
unzippedpath = RAWGISDIR+'/'+HydroShareID+'/'+HydroShareID+"/data/contents/"
onlyfiles = [f for f in listdir(unzippedpath) if f.endswith(".zip")]
HydroShareDownloadDIR = unzippedpath+onlyfiles[0].split('.')[0]    
sorted(listdir(HydroShareDownloadDIR))

['.DS_Store',
 'LULC30m.tif',
 'NLCD.tif',
 'NLCD.tif.aux.xml',
 'NLCD.tif.ovr',
 'dem30m.tif',
 'readme.txt',
 'rhessys',
 'rules',
 'swas_dem30.tfw',
 'swas_dem30.tif',
 'swas_dem30.tif.aux.xml',
 'swas_dem30.tif.ovr',
 'swas_dem30.tif.xml',
 'swas_gages_utm.dbf',
 'swas_gages_utm.prj',
 'swas_gages_utm.sbn',
 'swas_gages_utm.sbx',
 'swas_gages_utm.shp',
 'swas_gages_utm.shx']

<div class="alert alert-block alert-info">
Then, set file names below. <b>User edits are required.</b>
</div>

In [12]:
##--------------------------- set file names from the unzipped resources ---------------------------##
## please put in the local files names as the names above.
downloadedDEMfile = HydroShareDownloadDIR + '/' + 'dem30m.tif'
downloadedLULCfile = HydroShareDownloadDIR + '/' + 'NLCD.tif'
gageShapefile = HydroShareDownloadDIR + '/' + 'swas_gages_utm.shp' # (optional)

***
## 3) setup GRASS 7.4.x database

### 3.a import elevation from uploaded source

<div class="alert alert-block alert-info">
This block ask GRASS to read in raw elevation information and stored in RAW_elevation dataset. Skip this if you have previously done this step. Reprojection and re-sampling process would be done in the following blocks. 
</div>

In [13]:
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedDEMfile} output=demRAW location=elevationRAW

Starting GRASS GIS...
Executing <r.in.gdal -e --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/f52da10251c14c24bebfccbb32ccd614/f52da10251c14c24bebfccbb32ccd614/data/contents/SWAS_GIS/dem30m.tif output=demRAW location=elevationRAW> ...
Location <elevationRAW> created
Importing raster map <demRAW>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Default region for this location updated
Region for the current mapset updated
Execution of <r.in.gdal -e --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/f52da10251c14c24bebfccbb32ccd614/f52da10251c14c24bebfccbb32ccd614/data/contents/SWAS_GIS/dem30m.tif output=demRAW location=elevationRAW> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-success">
<b>Option 1:</b> re-projection and save the reprojected results to the RHESSys model GRASS dataset named after (RHESSysMODEL) that was defined by user at the beginning of the script. Take this option even if no reprojection is needed to import the RAW elevation data into the RHESSys model GRASS dataset
</div>

In [14]:
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region raster=demRAW
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.out.gdal --overwrite input=demRAW output={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o -e --overwrite input={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif output=dem
!grass74 {LOCATION}/{MAPSET} --exec g.region raster=dem

Starting GRASS GIS...
Executing <g.region raster=demRAW> ...
Execution of <g.region raster=demRAW> finished.
Cleaning up temporary files...
Starting GRASS GIS...
Executing <r.out.gdal --overwrite input=demRAW output=/scratch/hl8vq/jupyter_SWAS/raw_data/dem30m.tif format=GTiff> ...
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Float32>
Input raster map contains cells with NULL-value (no-data). The value -nan
will be used to represent no-data values in the input map. You can specify
a nodata value with the nodata option.
Exporting raster data to GTiff format...
ERROR 6: SetColorTable() only supported for Byte or UInt16 bands in TIFF format.
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File </scratch/hl8vq/jupyter_SWAS/raw_

<div class="alert alert-block alert-success">
<b>Option 2:</b> re-projection and re-cast spatial resolution. Then, save the reprojected results to the RHESSys model GRASS dataset named after (RHESSysMODEL) that was defined by user at the beginning of the script
</div>

In [ ]:
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region raster=demRAW
!grass74 {LOCATIONDEM}/{MAPSET} --exec g.region res={RESOLUTION} -a -p
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.resamp.stats -w input=demRAW output=dem{RESOLUTION}m 
!grass74 {LOCATIONDEM}/{MAPSET} --exec r.out.gdal --overwrite input=dem$RESOLUTION'm' output={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o -e --overwrite input={SCRATCH}/{PROJDIR}/raw_data/dem{RESOLUTION}m.tif output=dem
!grass74 {LOCATION}/{MAPSET} --exec g.region raster=dem

### 3.c import soil from uploaded source

<div class="alert alert-block alert-info">
SSURGO Soil data is often used to build RHESSys model. Note that there is no SSURGO data in the HydroShare resource for this catchment yet.
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> import from user downloaded SSURGO (not included in this example)
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={downloadedSSURGOdirectory}/spatial/soilmu_a_"$(echo $downloadedSSURGOdirectory | tr '[A-Z]' '[a-z]')".shp output=ssurgo location=soilRAW
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=soilRAW mapset=PERMANENT input=ssurgo output=ssurgo
!grass74 {LOCATION}/{MAPSET} --exec v.to.rast --overwrite input=ssurgo use=cat output=soil_ssurgo
!grass74 {LOCATION}/{MAPSET} --exec v.db.select --overwrite map=ssurgo separator=comma file={SCRATCH}/{PROJDIR}/raw_data/soil_cat_mukey.csv
## download R scripts to calculation soil types
!wget -O {RBASE}/ssurgo_extraction.R https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_extraction.R
!wget -O {RBASE}/ssurgo_soiltexture2gis.R https://raw.githubusercontent.com/laurencelin/ssurgo_extraction/master/ssurgo_soiltexture2gis.R
!Rscript {RBASE}/ssurgo_extraction.R {SCRATCH}/{PROJDIR}/raw_data/{downloadedSSURGOdirectory}
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/ssurgo_soiltexture2gis.R {SCRATCH}/{PROJDIR}/raw_data/soil_cat_mukey.csv {SCRATCH}/{PROJDIR}/raw_data/{downloadedSSURGOdirectory}/soil_mukey_texture.csv

<div class="alert alert-block alert-success">
<b>Option 2:</b> manually define by raster calculator. (see https://github.com/laurencelin/GIS2RHESSys/blob/master/soilCollection.csv) we take this option in this example.
</div>

In [15]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="soil_texture = 8"

Starting GRASS GIS...
Executing <r.mapcalc --overwrite expression=soil_texture = 8> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <r.mapcalc --overwrite expression=soil_texture = 8> finished.
Cleaning up temporary files...


### 3.d delinearate study catchment, construct drainage structures, and define some RHESSys variables

<div class="alert alert-block alert-info">
We need to define an outlet for a catchment
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> use Lat/Long (WSG84)
</div>

In [ ]:
%%bash -s {LOCATION} {MAPSET}
gageLat='38.250830' # catchment outlet WSG84 Lat (decimal degree)
gageLong='-78.748530' # catchment outlet WSG84 Long (decimal degree; includes the negative sign if applied)
declare $(grass74 $1/$2 --exec m.proj -i coordinates=$gageLong,$gageLat separator=space | awk '{print "xyCoord=" $1 "," $2}')
echo $xyCoord | grass74 $1/$2 --exec v.in.ascii in=- out=outlet x=1 y=2 separator=, --overwrite

<div class="alert alert-block alert-success">
<b>Option 2:</b> upload outlet.shp (we use this option in this example)
</div>

In [16]:
### ... if input is a shapefile point (import to LOCATIONOUTLET and the reproject to LOCATION as "outlet")
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={gageShapefile} output=outlet location=outletRAW
LOCATIONOUTLET = GISDBASE+'/'+"outletRAW"

Starting GRASS GIS...
Executing <v.in.ogr --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/f52da10251c14c24bebfccbb32ccd614/f52da10251c14c24bebfccbb32ccd614/data/contents/SWAS_GIS/swas_gages_utm.shp output=outlet location=outletRAW> ...
Location <outletRAW> created
Check if OGR layer <swas_gages_utm> contains polygons...
   0  20  40  60  80 100
Creating attribute table for layer <swas_gages_utm>...
Importing 5 features (OGR layer <swas_gages_utm>)...
   0  20  40  60  80 100
-----------------------------------------------------
Building topology for vector map <outlet@PERMANENT>...
Registering primitives...
5 primitives registered
5 vertices registered
Building areas...
   0  20  40  60  80 100
0 areas built
0 isles built
Attaching islands...
Attaching centroids...
  20  40  60  80 100
Number of nodes: 0
Number of primitives: 5
Number of points: 5
Number of lines: 0
Number of boundaries: 0
Number of centroids: 0
Number of areas: 0
Number of isles: 0
Execution of <v.in.ogr --over

<div class="alert alert-block alert-info">
<b>This step is optional:</b> The gage shapfile in this example contains multiple gage locations. So we need to identify a single location and extract it.
</div>

In [17]:
!grass74 {LOCATIONOUTLET}/{MAPSET} --exec v.extract --overwrite input=outlet type=point where="Site_ID = 'PAIN'" output=gage

Starting GRASS GIS...
Executing <v.extract --overwrite input=outlet type=point where=Site_ID = 'PAIN' output=gage> ...
Extracting features...
  20  40  60  80 100
Building topology for vector map <gage@PERMANENT>...
Registering primitives...
One primitive registered
One vertex registered
Building areas...
   0 100
0 areas built
0 isles built
Attaching islands...
Attaching centroids...
 100
Number of nodes: 0
Number of primitives: 1
Number of points: 1
Number of lines: 0
Number of boundaries: 0
Number of centroids: 0
Number of areas: 0
Number of isles: 0
Writing attributes...
Execution of <v.extract --overwrite input=outlet type=point where=Site_ID = 'PAIN' output=gage> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-info">
Reproject the identified gage location from RAW to the RHESSys model GRASS dataset named after (RHESSysMODEL) that was defined by user at the beginning of the script
</div>

In [18]:
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=outletRAW mapset=PERMANENT input=gage output=outlet

Starting GRASS GIS...
Executing <v.proj --overwrite location=outletRAW mapset=PERMANENT input=gage output=outlet> ...
Reprojecting primitives ...
Building topology for vector map <outlet@PERMANENT>...
Registering primitives...
One primitive registered
One vertex registered
Building areas...
   0 100
0 areas built
0 isles built
Attaching islands...
Attaching centroids...
 100
Number of nodes: 0
Number of primitives: 1
Number of points: 1
Number of lines: 0
Number of boundaries: 0
Number of centroids: 0
Number of areas: 0
Number of isles: 0
Execution of <v.proj --overwrite location=outletRAW mapset=PERMANENT input=gage output=outlet> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-info">
Catchment delineation (this step could be repeated to find the correct catchment.)
</div>


User needs to provide 
- the expected catchment size in terms of meter squares.  
- marginal error of the expected catchment size
- threshold value (number of grid cells) to define stream network 


In [19]:
expectedDrainageArea=10000000 # meter squre 

GRASS_thres = 1000 # grid cell
GRASS_drainarea_lowerbound = 0.7*expectedDrainageArea/RESOLUTION/RESOLUTION # (recommend 2% error)
GRASS_drainarea_upperbound = 1.3*expectedDrainageArea/RESOLUTION/RESOLUTION # (recommend 2% error)
!wget -O {RBASE}/grass_delineation.sh https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/grass_delineation.sh
!grass74 {LOCATION}/{MAPSET} --exec bash {RBASE}/grass_delineation.sh {GRASS_thres} {GRASS_drainarea_lowerbound} {GRASS_drainarea_upperbound}



--2019-03-10 21:25:15--  https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/grass_delineation.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2005 (2.0K) [text/plain]
Saving to: ‘/scratch/hl8vq/jupyter_SWAS/RLIB/grass_delineation.sh’

/scratch/hl8vq/jupy 100%[===================>]   1.96K  --.-KB/s    in 0.004s  

2019-03-10 21:25:15 (544 KB/s) - ‘/scratch/hl8vq/jupyter_SWAS/RLIB/grass_delineation.sh’ saved [2005/2005]

Starting GRASS GIS...
Executing <bash /scratch/hl8vq/jupyter_SWAS/RLIB/grass_delineation.sh 1000 7777.77777778 14444.4444444> ...
ERROR: No existing MASK to remove
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42

In [20]:
!grass74 {LOCATION}/{MAPSET} --exec r.stats -a -c input=basin

Starting GRASS GIS...
Executing <r.stats -a -c input=basin> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
1 12699000.000000 14110
* 10474200.000000 11638
Execution of <r.stats -a -c input=basin> finished.
Cleaning up temporary files...


### 3.e define zones

<div class="alert alert-block alert-info">
<b>Note:</b> Zone is for the climate
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> One zone for the entire catchment
</div>

In [21]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="zone = hill"

Starting GRASS GIS...
Executing <r.mapcalc --overwrite expression=zone = hill> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <r.mapcalc --overwrite expression=zone = hill> finished.
Cleaning up temporary files...


</div>
<div class="alert alert-block alert-success">
<b>Option 2:</b> define zone by patch
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="zone = patch"

### 3.f import LULC from uploaded scource

<div class="alert alert-block alert-info">
Import the raw LULC to GRASS
</div>

In [22]:
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedLULCfile} output=lulcRAW location=lulcRAW

Starting GRASS GIS...
Executing <r.in.gdal -e --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/f52da10251c14c24bebfccbb32ccd614/f52da10251c14c24bebfccbb32ccd614/data/contents/SWAS_GIS/NLCD.tif output=lulcRAW location=lulcRAW> ...
Location <lulcRAW> created
Importing raster map <lulcRAW>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Default region for this location updated
Region for the current mapset updated
Execution of <r.in.gdal -e --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/f52da10251c14c24bebfccbb32ccd614/f52da10251c14c24bebfccbb32ccd614/data/contents/SWAS_GIS/NLCD.tif output=lulcRAW location=lulcRAW> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-info">
Reproject raw LULC to the RHESSys model GRASS dataset named after (RHESSysMODEL) that was defined by user at the beginning of the script
</div>

In [23]:
!grass74 {LOCATIONLULC}/{MAPSET} --exec r.out.gdal --overwrite input=lulcRAW output={SCRATCH}/{PROJDIR}/raw_data/LULC{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/LULC{RESOLUTION}m.tif output=LULCcode

Starting GRASS GIS...
Executing <r.out.gdal --overwrite input=lulcRAW output=/scratch/hl8vq/jupyter_SWAS/raw_data/LULC30m.tif format=GTiff> ...
Checking GDAL data type and nodata value...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
Using GDAL data type <Byte>
Exporting raster data to GTiff format...
   2   5   8  11  14  17  20  23  26  29  32  35  38  41  44  47  50  53  56  59  62  65  68  71  74  77  80  83  86  89  92  95  98 100
r.out.gdal complete. File
</scratch/hl8vq/jupyter_SWAS/raw_data/LULC30m.tif> created.
Execution of <r.out.gdal --overwrite input=lulcRAW output=/scratch/hl8vq/jupyter_SWAS/raw_data/LULC30m.tif format=GTiff> finished.
Cleaning up temporary files...
Starting GRASS GIS...
Executing <r.in.gdal -o --overwrite input=/scratch/hl8vq/jupyter_SWAS/raw_data/LULC30m.tif output=LULCcode> ...
Over-riding projection check
Importing raster map <LULCcode>...
   0   3   6   9  12  1

<div class="alert alert-block alert-info">
Listing LULC indexes/IDs/codes that are in the catchment 
</div>

In [24]:
!grass74 {LOCATION}/{MAPSET} --exec r.stats input=LULCcode -c

Starting GRASS GIS...
Executing <r.stats input=LULCcode -c> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
21 323
22 4
31 11
41 13426
42 172
43 174
* 11638
Execution of <r.stats input=LULCcode -c> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-info">
Define RHESSys stratum objects and landuse classes based on the LULC indexees/IDs/codes above. Note that we define one stratum object per patch in the model.
</div>
- Stratum information (see https://github.com/laurencelin/GIS2RHESSys/blob/master/vegCollection.csv)
    - stratum vegetation ID (users define)
    - stratum cover fraction (users define)
    - stratum LAI (users define)
- Landuse object
    - landuse type (ID) (users define) that associates with the following information (see https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/soilCollection.csv): 
        - surface detention storage capacity 
        - annual areal fertilizer rate

In [33]:
%%writefile {RBASE}/LULC2RHESSys.R
options(scipen=999)
library(rgrass7)
rast = readRAST(c('basin','str','LULCcode'))
mask = !is.na(rast@data[[1]])
str = rast@data[[2]][mask]
lulc = rast@data[[3]][mask]

rast$vegid = rep(NA,length(mask))
rast$landuse = rep(NA,length(mask))
rast$lai = rep(NA,length(mask))
rast$impervious = rep(NA,length(mask))
rast$coverFrac = rep(NA,length(mask))

#--- forest
    cond=(lulc==41 | lulc==43 | lulc==90 | lulc==95) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(2) # deciduous @ vegCollection.csv
    rast$landuse[mask][cond] = as.integer(2) # undeveloped @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 4.5
    rast$impervious[mask][cond] = 0.0

    cond=(lulc==42) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(1) # evergreen @ vegCollection.csv 
    rast$landuse[mask][cond] = as.integer(2) # undeveloped @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 5.0
    rast$impervious[mask][cond] = 0.0

    cond=(lulc==51 | lulc==52) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(6) # shrub @ vegCollection.csv 
    rast$landuse[mask][cond] = as.integer(2) # undeveloped @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 2
    rast$impervious[mask][cond] = 0.0

#--- pasture / grass / agriculture 
    cond=(lulc==71 | lulc==72 | lulc==81 | lulc==82) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(3) # grass @ vegCollection.csv 
    rast$landuse[mask][cond] = as.integer(1) # grass @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 1.5
    rast$impervious[mask][cond] = 0.0

#--- urban / barren
    cond=(lulc==31) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(6) # shrub @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(2) # undeveloped @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 0.15 # 15% grass/lawn
    rast$lai[mask][cond] = 3.0
    rast$impervious[mask][cond] = 0.85

    cond=(lulc==21) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(3) # grass @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(3) # urban @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 0.8 # 80% grass/lawn
    rast$lai[mask][cond] = 1.5
    rast$impervious[mask][cond] = 0.2

    cond=(lulc==22) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(3) # grass @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(3) # urban @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 0.5 # 50% grass/lawn
    rast$lai[mask][cond] = 1.5
    rast$impervious[mask][cond] = 0.5

    cond=(lulc==23) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(3) # grass @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(3) # urban @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 0.2 # 20% grass/lawn
    rast$lai[mask][cond] = 1.5
    rast$impervious[mask][cond] = 0.8

    cond=(lulc==24) & is.na(str); 
    rast$vegid[mask][cond] = as.integer(4) # no-veg @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(3) # urban @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 1.0
    rast$impervious[mask][cond] = 1.0

    cond= !is.na(str); 
    rast$vegid[mask][cond] = as.integer(4) # no-veg @ vegCollection.csv (may vary by state/county/city/local)
    rast$landuse[mask][cond] = as.integer(2) # undeveloped @ lulcCollection.csv
    rast$coverFrac[mask][cond] = 1.0
    rast$lai[mask][cond] = 0.0
    rast$impervious[mask][cond] = 0.0

writeRAST(rast,'vegid',zcol='vegid',overwrite=T)
writeRAST(rast,'landuse',zcol='landuse',overwrite=T)
writeRAST(rast,'coverFrac',zcol='coverFrac',overwrite=T)
writeRAST(rast,'lai',zcol='lai',overwrite=T)
writeRAST(rast,'impervious',zcol='impervious',overwrite=T)

Overwriting /scratch/hl8vq/jupyter_SWAS/RLIB/LULC2RHESSys.R


In [34]:
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/LULC2RHESSys.R

Starting GRASS GIS...
Executing <Rscript /scratch/hl8vq/jupyter_SWAS/RLIB/LULC2RHESSys.R> ...
Loading required package: sp
Loading required package: XML
GRASS GIS interface loaded with GRASS version: GRASS 7.4.3 (2018)
and location: PAIN
Creating BIL support files...
Exporting raster as integer values (bytes=4)
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Creating BIL support files...
Exporting raster as integer values (bytes=4)
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Creating BIL support files...
Exporting raster as integer values (bytes=4)
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  

In [35]:
!grass74 {LOCATION}/{MAPSET} --exec g.list -p type='rast' | cat

Starting GRASS GIS...
Executing <g.list -p type=rast> ...
----------------------------------------------
raster files available in mapset <PERMANENT>:
LULCcode        colmap          isohyet         slope_          wetness_index
MASK            coverFrac       lai             soil_texture    xmap
ONE             dem             landuse         str             ymap
ZERO            drain           patch           sub             zone
aspect          east_000        roads           uaa
aspect_         hill            rowmap          vegid
basin           impervious      slope           west_180

Execution of <g.list -p type=rast> finished.
Cleaning up temporary files...


### 3.h Road network / Isohyet

<div class="alert alert-block alert-info">
Define road network. Note that we must define a road raster map for w2g and flowtable calculation. So define a null road raster (below) if there is no road.
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> make an empty road map.
</div>

In [29]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="roads = null()"

Starting GRASS GIS...
Executing <r.mapcalc --overwrite expression=roads = null()> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <r.mapcalc --overwrite expression=roads = null()> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-success">
<b>Option 2:</b> upload road.shp and the rasterize it
</div>

In [ ]:
!grass74 {LOCATION}/{MAPSET} --exec v.in.ogr --overwrite input={SCRATCH}/{PROJDIR}/raw_data/{downloadedROADfile} output=roads location=roadRAW
!grass74 {LOCATION}/{MAPSET} --exec v.proj --overwrite location=roadRAW mapset=PERMANENT input=roads output=roads
!grass74 {LOCATION}/{MAPSET} --exec v.to.rast --overwrite input=roads@PERMANENT output=roads use=cat

<div class="alert alert-block alert-success">
<b>Option 3:</b> upload road raster (we use this option in this example)
</div>

In [ ]:
downloadedROADfile = HydroShareDownloadDIR + '/' + 'roads.tif'
LOCATIONROAD = GISDBASE+'/'+'roadRAW'
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedROADfile} output=roadRAW location=roadRAW
!grass74 {LOCATIONROAD}/{MAPSET} --exec r.out.gdal --overwrite input=roadRAW output={SCRATCH}/{PROJDIR}/raw_data/ROAD{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/ROAD{RESOLUTION}m.tif output=roads

<div class="alert alert-block alert-info">
Define isohyet. Note that we must define a road raster map for w2g and flowtable calculation. 
</div>

<div class="alert alert-block alert-success">
<b>Option 1:</b> no isohyet
</div>

In [30]:
!grass74 {LOCATION}/{MAPSET} --exec r.mapcalc --overwrite expression="isohyet = 1"

Starting GRASS GIS...
Executing <r.mapcalc --overwrite expression=isohyet = 1> ...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Execution of <r.mapcalc --overwrite expression=isohyet = 1> finished.
Cleaning up temporary files...


<div class="alert alert-block alert-success">
<b>Option 2:</b> import isohyet raster
</div>

In [ ]:
downloadedISOHYETfile = HydroShareDownloadDIR + '/' + 'isohyet.tif'
LOCATIONISOHYET = GISDBASE+'/'+'isohyetRAW'
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -e --overwrite input={downloadedISOHYETfile} output=isohyetRAW location=isohyetRAW
!grass74 {LOCATIONISOHYET}/{MAPSET} --exec r.out.gdal --overwrite input=isohyetRAW output={SCRATCH}/{PROJDIR}/raw_data/ISOHYET{RESOLUTION}m.tif format=GTiff
!grass74 {LOCATION}/{MAPSET} --exec r.in.gdal -o --overwrite input={SCRATCH}/{PROJDIR}/raw_data/ISOHYET{RESOLUTION}m.tif output=isohyet

***
## 4) constructing worldfile and flowtable to RHESSys

<div class="alert alert-block alert-success">
<b>Option1:</b> copy climate series data from HydroShare download (we use this option in this example)
</div>

In [31]:
from os import listdir 
!cp -r {HydroShareDownloadDIR}/rhessys/clim {SCRATCH}/{PROJDIR}/{RHESSysMODEL}
onlyfiles = [f for f in listdir(SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/clim') if f.endswith(".base")]
tmp = !head -n1 {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/clim/{onlyfiles[0]}
climateBaseFile = 'clim/'+onlyfiles[0]
climateBaseID = tmp.fields(0)[0]
print climateBaseFile, climateBaseID

clim/exClimData.base 101


In [36]:
!wget -O {RBASE}/g2w.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/g2w.R
!wget -O {RBASE}/vegCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/vegCollection.csv
!wget -O {RBASE}/soilCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/soilCollection.csv
!wget -O {RBASE}/lulcCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulcCollection.csv
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/g2w.R {SCRATCH}/{PROJDIR} {climateBaseID} {climateBaseFile} {RBASE}/vegCollection.csv {RBASE}/soilCollection.csv {RBASE}/lulcCollection.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.hdr {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/defs

--2019-03-10 21:35:46--  https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/g2w.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976 (17K) [text/plain]
Saving to: ‘/scratch/hl8vq/jupyter_SWAS/RLIB/g2w.R’

/scratch/hl8vq/jupy 100%[===================>]  16.58K  --.-KB/s    in 0.002s  

2019-03-10 21:35:46 (7.47 MB/s) - ‘/scratch/hl8vq/jupyter_SWAS/RLIB/g2w.R’ saved [16976/16976]

--2019-03-10 21:35:46--  https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/vegCollection.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12483 (12K) [text/plain]
Saving to: ‘/scratch/hl8vq/jupyter

<div class="alert alert-block alert-success">
<b>Option2:</b> define climate info by user
</div>

In [ ]:
ls -l {RAWGISDIR}/rhessys/clim

In [ ]:
climateBaseFile = 'clim/exClimData.base'
climateBaseID = 101    

!wget -O {RBASE}/g2w.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/g2w.R
!wget -O {RBASE}/vegCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/vegCollection.csv
!wget -O {RBASE}/soilCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/soilCollection.csv
!wget -O {RBASE}/lulcCollection.csv https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/lulcCollection.csv
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/g2w.R {SCRATCH}/{PROJDIR} {climateBaseID} {climateBaseFile} {RBASE}/vegCollection.csv {RBASE}/soilCollection.csv {RBASE}/lulcCollection.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.hdr {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/defs

<div class="alert alert-block alert-info">
convert extracted worldfile.csv to RHESSys worldfile format.
</div>

In [37]:
!wget -O {RBASE}/LIB_RHESSys_writeTable2World.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/LIB_RHESSys_writeTable2World.R
!Rscript {RBASE}/LIB_RHESSys_writeTable2World.R na {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile.csv {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles/worldfile


--2019-03-10 21:44:20--  https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/LIB_RHESSys_writeTable2World.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18715 (18K) [text/plain]
Saving to: ‘/scratch/hl8vq/jupyter_SWAS/RLIB/LIB_RHESSys_writeTable2World.R’

/scratch/hl8vq/jupy 100%[===================>]  18.28K  --.-KB/s    in 0.004s  

2019-03-10 21:44:20 (4.89 MB/s) - ‘/scratch/hl8vq/jupyter_SWAS/RLIB/LIB_RHESSys_writeTable2World.R’ saved [18715/18715]

[1] "header = na"
[1] "basefile = /scratch/hl8vq/jupyter_SWAS/PAIN/worldfiles/worldfile.csv"
[1] "outputfile = /scratch/hl8vq/jupyter_SWAS/PAIN/worldfiles/worldfile"


<div class="alert alert-block alert-info">
(subsurface) flow table calculation 
</div>

In [42]:
!wget -O {RBASE}/createFlowRouting.R https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/createFlowRouting.R
!grass74 {LOCATION}/{MAPSET} --exec Rscript {RBASE}/createFlowRouting.R {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/flows/flowtable.txt

--2019-03-10 21:52:24--  https://raw.githubusercontent.com/laurencelin/GIS2RHESSys/master/createFlowRouting.R
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11172 (11K) [text/plain]
Saving to: ‘/scratch/hl8vq/jupyter_SWAS/RLIB/createFlowRouting.R’

/scratch/hl8vq/jupy 100%[===================>]  10.91K  --.-KB/s    in 0s      

2019-03-10 21:52:24 (74.2 MB/s) - ‘/scratch/hl8vq/jupyter_SWAS/RLIB/createFlowRouting.R’ saved [11172/11172]

Starting GRASS GIS...
Executing <Rscript /scratch/hl8vq/jupyter_SWAS/RLIB/createFlowRouting.R /scratch/hl8vq/jupyter_SWAS/PAIN/flows/flowtable.txt> ...
Loading required package: sp
Loading required package: XML
GRASS GIS interface loaded with GRASS version: GRASS 7.4.3 (2018)
and location: PAIN
rgdal: version: 1.3-6, (SVN revision 773)
 Geospatial Data Abstraction

<div class="alert alert-block alert-info">
verfied output files.
</div>

In [43]:
ls -lh {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/flows

total 4.1M
-rw-r--r-- 1 hl8vq users 4.1M Mar 10 21:53 flowtable.txt


In [40]:
ls -lh {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/worldfiles

total 66M
-rw-r--r-- 1 hl8vq users  56M Mar 10 21:46 worldfile
-rw-r--r-- 1 hl8vq users 9.9M Mar 10 21:36 worldfile.csv
-rw-r--r-- 1 hl8vq users  717 Mar 10 21:36 worldfile.hdr


In [41]:
ls -l {SCRATCH}/{PROJDIR}/grass_dataset/

total 16
drwxr-xr-x 3 hl8vq users 4096 Mar 10 21:22 elevationRAW/
drwxr-xr-x 3 hl8vq users 4096 Mar 10 21:28 lulcRAW/
drwxr-xr-x 3 hl8vq users 4096 Mar 10 21:24 outletRAW/
drwxr-xr-x 3 hl8vq users 4096 Mar 10 21:20 PAIN/


****
***
***
## 5) model parameters

### 1) -s

-s : value1, value2, value3 
  * the m, K, and soil depth parameter value initialized for each patch in the worldfile are multiplied by 1) value1 2) value2 and 3) value3 respectively during a simulation.)

 - value1 : m (the decay of hydraulic conductivity with depth)
 
 - value2 : K (hydraulic conductivity at the surface)
 
 - value3 : soil depth (hydraulic conductivity at the surface)

### 2) -sv

-sv : value1, value2 
* (the m, K are multipliers to scale the vertical decay of hydraulic conductivity with depth (m), and vertical hydraulic conductivity at the surface (K).

### 3) -gw

-gw : value1, value2 
 * value1 : The first value is a multiplier on the sat_to_gw_coeff parameter set in the soil definition file (representing the amount of water moving from the saturated store to the groundwater store).
  - sat_to_gw_coeff(%) : the amount of water moving from the saturated store to the groundwater store; bypasses roots
 * value2 : The second value is a multiplier on the gw_loss_coeff parameter in the hillslope default file (representing the amount of water moving from the groundwater store to the stream).
  - gw_loss_coeff(%) : Percent of groundwater store lost to stream

<div class="alert alert-block alert-info">
Set soil sensitivity parameters of RHESSys Model
</div>

In [44]:
start_date = '2000 1 1 1'

In [45]:
end_date = '2008 10 1 1'

In [67]:
RHESSysOUTPUTDIR = 'output'
!mkdir {SCRATCH}/{PROJDIR}/{RHESSysMODEL}/{RHESSysOUTPUTDIR}

In [47]:
# -b only basin output; -gwtoriparian receiving groundwater and put in stream
RHESSys_flags = "-b -newcaprise -capr 0.001 -capMax 0.01 -slowDrain"
RHESSys_worldfile = "-w worldfiles/worldfile -whdr worldfiles/worldfile.hdr"
RHESSys_flowtable = "-r flows/flowtable.txt"
RHESSys_tecfile = "-t tecfiles/tec_daily.txt"
RHESSys_output = '-pre '+RHESSysOUTPUTDIR+'/rhessys'

<div class="alert alert-block alert-info">
set a tecfile
</div>

In [48]:
tecfile = SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/tecfiles/tec_daily.txt'

In [49]:
%%writefile {tecfile}
2000 1 1 1 print_daily_on

Writing /scratch/hl8vq/jupyter_SWAS/PAIN/tecfiles/tec_daily.txt


<div class="alert alert-block alert-info">
download and compile RHESSys code
</div>

In [55]:
RHESSysCODE = SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/code/rhessysCODE.zip'
!wget -O {RHESSysCODE} https://github.com/laurencelin/RHESSys5.20.EastForest/raw/Jan13_2019/rhessys5.20.zip

--2019-03-10 21:57:21--  https://github.com/laurencelin/RHESSys5.20.EastForest/raw/Jan13_2019/rhessys5.20.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/laurencelin/RHESSys5.20.EastForest/Jan13_2019/rhessys5.20.zip [following]
--2019-03-10 21:57:22--  https://raw.githubusercontent.com/laurencelin/RHESSys5.20.EastForest/Jan13_2019/rhessys5.20.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1316668 (1.3M) [application/zip]
Saving to: ‘/scratch/hl8vq/jupyter_SWAS/PAIN/code/rhessysCODE.zip’

/scratch/hl8vq/jupy 100%[===================>]   1.25M  --.-KB/s    in 0.07s   

2019-03-10 21:57:22 (17.2 MB/s) - ‘/sc

In [57]:
import zipfile
with zipfile.ZipFile(RHESSysCODE, 'r') as zip_ref:
    zip_ref.extractall(SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/code')

In [58]:
%%bash -s {SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/code/rhessys5.20/rhessys'}
cd $1
make 

mkdir -p objects
gcc  -c -Wall -g -std=c99 -I include init/read_netcdf_dummy.c -o objects/read_netcdf.o
gcc  -c -Wall -g -std=c99 -I include cn/Ksat_z_curve.c -o objects/Ksat_z_curve.o
gcc  -c -Wall -g -std=c99 -I include output/add_growth_headers.c -o objects/add_growth_headers.o
gcc  -c -Wall -g -std=c99 -I include output/add_headers.c -o objects/add_headers.o
gcc  -c -Wall -g -std=c99 -I include util/alloc.c -o objects/alloc.o
gcc  -c -Wall -g -std=c99 -I include cn/allocate_annual_growth.c -o objects/allocate_annual_growth.o
gcc  -c -Wall -g -std=c99 -I include cn/allocate_daily_growth.c -o objects/allocate_daily_growth.o
gcc  -c -Wall -g -std=c99 -I include init/assign_base_station.c -o objects/assign_base_station.o
gcc  -c -Wall -g -std=c99 -I include init/assign_base_station_xy.c -o objects/assign_base_station_xy.o
gcc  -c -Wall -g -std=c99 -I include init/assign_neighbours.c -o objects/assign_neighbours.o
gcc  -c -Wall -g -std=c99 -I include cycle/basin_daily_F.c -o objects/bas

output/add_growth_headers.c: In function ‘add_growth_headers’:
output/add_growth_headers.c:38:6: warning: variable ‘check’ set but not used [-Wunused-but-set-variable]
  int check;
      ^
output/add_headers.c: In function ‘add_headers’:
output/add_headers.c:41:6: warning: variable ‘check’ set but not used [-Wunused-but-set-variable]
  int check;
      ^
cn/allocate_annual_growth.c: In function ‘allocate_annual_growth’:
cn/allocate_annual_growth.c:99:16: warning: format ‘%d’ expects argument of type ‘int’, but argument 4 has type ‘long int’ [-Wformat=]
         printf("report,%d,%d,%d,%d,%d,%d,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e,%e\n",
                ^
cn/allocate_annual_growth.c:99:16: warning: format ‘%d’ expects argument of type ‘int’, but argument 5 has type ‘long int’ [-Wformat=]
cn/allocate_annual_growth.c:99:16: warning: format ‘%d’ expects argument of type ‘int’, but argument 6 has type ‘long int’ [-Wformat=]
cn/allocate_annual_growth.c:84:9: warning: unused variab

In [59]:
!ls -l {SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/code/rhessys5.20/rhessys'}

total 5200
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 clim
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 cn
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 cycle
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 hydro
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 include
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 init
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 lib
-rw-r--r-- 1 hl8vq users   17378 Mar 10 21:58 main.c
-rw-r--r-- 1 hl8vq users   55624 Mar 10 21:58 makefile
drwxr-xr-x 2 hl8vq users   20480 Mar 10 22:00 objects
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 output
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 rad
-rwxr-xr-x 1 hl8vq users 5175256 Mar 10 22:00 rhessys5.20.0.develop
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 tec
drwxr-xr-x 3 hl8vq users    4096 Mar 10 21:58 test
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:58 util


<div class="alert alert-block alert-info">
copy/move the compiled RHESSys binary executable file, i.e., rhessys5.20.0.develop in our example, to the RHESSys model directory
</div>

In [60]:
!cp {SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/code/rhessys5.20/rhessys/rhessys5.20.0.develop'} {SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL}

In [61]:
!ls -l {SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL}

total 4120
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:29 clim
drwxr-xr-x 4 hl8vq users    4096 Mar 10 21:58 code
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:36 defs
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:52 flows
-rwxr-xr-x 1 hl8vq users 5175256 Mar 10 22:04 rhessys5.20.0.develop
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:53 tecfiles
drwxr-xr-x 2 hl8vq users    4096 Mar 10 21:44 worldfiles


<div class="alert alert-block alert-info">
set a list of RHESSys model runs with randomized parameters for model calibration. These model runs are then submitted to Rivanna computating system. Note that DO NOT run the models within the Jupyter notebook.
</div>

<div class="alert alert-block alert-info">
First we prepared a SLURM job submission script. Mark down the RHESSys binary executable file name below.
</div>

In [62]:
jobscriptfile = SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/Rivanna_std.sh'

In [63]:
%%writefile {jobscriptfile}
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH -t 144:00:00
#SBATCH -p standard
./rhessys5.20.0.develop $v

Writing /scratch/hl8vq/jupyter_SWAS/PAIN/Rivanna_std.sh


<div class="alert alert-block alert-info">
Then we set parameter boundaries. command numpy.arange(1,11) will set iteration index from 1 to 10
</div>

In [64]:
import numpy
ITR = numpy.arange(1,11)
s_value1 = numpy.random.uniform(0.001,20,ITR.size)
s_value2 = numpy.random.uniform(0.1,300,ITR.size)
sv_value1 = numpy.random.uniform(0.001,20,ITR.size)
sv_value2 = numpy.random.uniform(0.1,300,ITR.size)
gw_value1 = numpy.random.uniform(0.001,0.2,ITR.size)
gw_value2 = numpy.random.uniform(0.001,0.2,ITR.size)

- JOBID is for job tracking on Rivanna
- parallelrun101.sh is a shell/bash script file for job sbumissions. User can change the name of this shell/bash script.


In [65]:
JOBID = '12345'
parallelrunfile = SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL+'/parallelrun101.sh'
with open(parallelrunfile, "w") as f:
    f.write("#!/bin/bash\n")
    for i in numpy.arange(0,ITR.size):
        cmd = "sbatch -o {}/log.txt -J p{} --export=v=\'-st {} -ed {} {} {} {} {} {} -s {} {} -sv {} {} -gw {} {}\' Rivanna_std.sh\n".format(
            RHESSysOUTPUTDIR, JOBID,
            start_date, end_date, 
            RHESSys_flags, RHESSys_tecfile,
            RHESSys_worldfile, RHESSys_flowtable,
            RHESSys_output+str(ITR[i]),
            s_value1[i], s_value2[i],
            sv_value1[i], sv_value2[i],
            gw_value1[i], gw_value2[i])
        #print cmd
        f.write(cmd)

<div class="alert alert-block alert-info">
submit jobs to Rivanna computating system. Note that DO NOT run the models within the Jupyter notebook.
</div>

Job submission to Rivanna cannot be done within Jupyter notebook (at the moment). Perphas it's the policy for Jupyter notebook on Rivanna.

sbatch and squeue commands cannot be found.

In [66]:
#parallelrunLocation = SCRATCH+'/'+PROJDIR+'/'+RHESSysMODEL

In [68]:
#%%bash -s {parallelrunLocation} {parallelrunfile}
#cd $1
#sh $2

/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 2: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 3: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 4: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 5: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 6: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 7: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 8: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found
/scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: 9: /scratch/hl8vq/jupyter_SWAS/PAIN/parallelrun101.sh: sbatch: not found


In [70]:
%%bash -s {parallelrunLocation}
cd $1
sbatch -o output/log.txt -J p12345 --export=v='-st 2000 1 1 1 -ed 2008 10 1 1 -b -newcaprise -capr 0.001 -capMax 0.01 -slowDrain -t tecfiles/tec_daily.txt -w worldfiles/worldfile -whdr worldfiles/worldfile.hdr -r flows/flowtable.txt -pre output/rhessys1 -s 6.19316358302 185.715352351 -sv 11.7003472176 209.310088455 -gw 0.0732621071084 0.156810828103' Rivanna_std.sh

bash: line 2: sbatch: command not found


<div class="alert alert-block alert-info">
check the job status
</div>

In [69]:
#!squeue -u {USER} -o %t | sort | uniq -c

/bin/sh: 1: squeue: not found


In [ ]:
Laurence edits end here.

## 5) Plotting of RHESSys Model Output (codes copied from previous scripts;)

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

 ## Basin Daily Output
 |                    RHESSys Output Abbreviation                   | Description |   Units |   
 |---------------------------------------------|-------------|-------------------|
 |         pot_surface_infil| Rain Throughfall    | mm          | 
 |       snow_thr        | Snow Throughfall    | mm         | 
 |sat_def_z  | Saturation Deficit with depth    | mm of depth          | 
 |sat_def | Saturation Deficit - volume  | mm of water       | 
 |rz_storage	|Rooting Zone Storage	|mm of water
 |unsat_stor|	Unsaturated Storage	|mm
 |rz_drainage|	Rooting Zone Drainage|	mm
 |unsat_drain|	Unsaturated| Storage	mm
 |cap	|Capillary Rise|	mm
 |evap	|Evaporation|	mm
 |snowpack	|Snow Water Equivalent (SWE)|	mm
 |trans	|Transpiration|	mm
 |baseflow	|Baseflow	|mm
 |return	|Return flow|	mm
 |streamflow	|Total Stream Outflow|	mm (normalized by basin area)
 | psn	|Net Photosynthesis	|kgC/m2
|lai	|Leaf Area Index	|m2/m2
|gw.Qout	|Groundwater Output	|mm
|gw.storage	|Groundwater Store	|mm
|detention_store|	Detention Store	|mm
|%sat_area|	Percent Saturated Area	|m2/m2
|litter_store|	Litter intercepted water Store	|m2/m2
|canopy_store|	Canopy Intercepted water Store	|m2/m2
|%snow_cover|	Percent Snow Cover	|m2/m2
|snow_subl|	Snow Sublimation	|
|trans_var|	Spatial variation in transpiration	|
|acc_trans		||
|acctransv_var		||
|pet	|Potential Evapotranspiration|	mm
|dC13		||
|precip	|Precipitation|	mm
|pcp_assim||		
|mortf	|Fraction of Basin that have tree mortality	|
|tmax	|Maximum Temperature	|°C
|tmin	|Minimum Temperature	|°C
|tavg	|Average Temperature	|°C
|vpd	|Vapor Pressure Deficit	|Pa
|snowfall	|Snowfall	|
|recharge	|_Recharge of water to soil	|
|gpsn	|Gross Photosynthesis	|kgC/m2
|resp	|_ Respiration_	|kgC/m2
|gs	|Canopy Conductance	|mm/s?
|rootdepth	|Rooting depth	|
|plantc	|Plant Carbon	|kgC/m2
|snowmelt	|Snow Melt	|
|canopysubl	|Canopy Sublimation	|
|routedstreamflow	||	
|canopy_snow	|Snow Intercepted on Canopy	|
|height	|Canopy height	|
|evap_can	|Canopy Evaporation?	|
|evap_lit	|Litter Evaporation_	|
|evap_soil	|Soil Evaporation_	|
|litrc	|Litter Carbon_	|
|Kdown	|Downward (from atmosphere) Direct Shortwave Radiation_	|
|Ldown	|Downward (from atmosphere) Longwave Radiation_	|
|Kup	|Reflected (upward) Shortwave Radiation_	|
|Lup	|Reflected (upward) Longwave Radiation_	|
|Kstar_can	|Absorbed shortwave by canopy	|
|Kstar_soil	|Absorbed shortwave by soil	|
|Kstar_snow	|Absorbed shortwave bysnow	|
|Lstar_can	|Absorbed longwave by canopy	|
|Lstar_soil	|Absorbed longwave by soil	|
|Lstar_snow	|Absorbed longwave by snow	|
|LE_canopy	|Latent heat evaporated by canopy	|
|LE_soil	La	||
|LE_snow		||
|Lstar_strat		||
|canopydrip		||
|ga	|Aerodynamic Conductance	|mm/s

In [ ]:
path = "output/"
#basin_daily_output = pd.read_csv(path + output_prefix +"_basin.daily", delimiter=" ")
basin_daily_output = pd.read_csv(path + "rhessys_m6K3_basin.daily", delimiter=" ")

In [ ]:
start_date = "2000-01-01"
end_date = "2001-09-30"
#end_date = "2008-09-30"

In [ ]:
date_index = pd.date_range(start_date, end_date, freq='1D')
basin_daily_output_date = basin_daily_output.insert(loc=0, column='Date', value=date_index.values)
#basin_daily_output_date1 = basin_daily_output.insert1(loc=0, column='Date', value=date_index.values)
basin_daily_output_date_index = basin_daily_output.set_index('Date')
#basin_daily_output_date1_index = basin_daily_output1.set_index('Date')

In [ ]:
basin_daily_output_date_index.head()

In [ ]:
plt_start_date = '2001-01-01'
plt_end_date = '2001-09-30'

In [ ]:
basin_daily_output_f = basin_daily_output_date_index.loc[plt_start_date:plt_end_date]
basin_daily_output1_f = basin_daily_output_date1_index.loc[plt_start_date:plt_end_date]

In [ ]:
# Rain Throughfall (mm)
basin_daily_output_f['pot_surface_infil'].plot(figsize=(17,5))

In [ ]:
# Saturation Deficit with depth (mm of depth)
basin_daily_output_f['sat_def_z'].plot(figsize=(17,5))
basin_daily_output1_f['sat_def_z'].plot(figsize=(17,5))

In [ ]:
# Saturation Deficit - volume (mm of water)
basin_daily_output_f['sat_def'].plot(figsize=(17,5))

In [ ]:
# Rooting Zone Storage  (mm of water)
basin_daily_output_f['rz_storage'].plot(figsize=(17,5))

In [ ]:
# Unsaturated Storage (mm)
basin_daily_output_f['unsat_stor'].plot(figsize=(17,5))

In [ ]:
# Rooting Zone Drainage (mm)
basin_daily_output_f['rz_drainage'].plot(figsize=(17,5))

In [ ]:
# Unsaturated Drainage (mm)
basin_daily_output_f['unsat_drain'].plot(figsize=(17,5))

In [ ]:
# Capillary Rise (mm)
basin_daily_output_f['cap'].plot(figsize=(17,5))

In [ ]:
# Evaporation (mm)
basin_daily_output_f['evap'].plot(figsize=(17,5))

In [ ]:
# Snow Water Equivalent (SWE) (mm)
basin_daily_output_f['snowpack'].plot(figsize=(17,5))

In [ ]:
# Transpiration (mm)
basin_daily_output_f['trans'].plot(figsize=(17,5))

In [ ]:
# Baseflow (mm)
basin_daily_output_f['baseflow'].plot(figsize=(17,5))

In [ ]:
# Return flow (mm)
basin_daily_output_f['return'].plot(figsize=(17,5))

In [ ]:
# Total Stream Outflow (mm (normalized by basin area))
basin_daily_output_f['streamflow'].plot(figsize=(17,5))

In [ ]:
# Net Photosynthesis (kgC/m2)
basin_daily_output_f['psn'].plot(figsize=(17,5))

In [ ]:
# Leaf Area Index (m2/m2)
basin_daily_output_f['lai'].plot(figsize=(17,5))

In [ ]:
# Groundwater Output (mm)
basin_daily_output_f['gw.Qout'].plot(figsize=(17,5))

In [ ]:
# Groundwater Store (mm)
basin_daily_output_f['gw.storage'].plot(figsize=(17,5))

In [ ]:
# Detention Store (mm)
basin_daily_output_f['detention_store'].plot(figsize=(17,5))

In [ ]:
# Percent Saturated Area (m2/m2)
basin_daily_output_f['%sat_area'].plot(figsize=(17,5))

In [ ]:
# Litter intercepted water Store (m2/m2)
basin_daily_output_f['litter_store'].plot(figsize=(17,5))

In [ ]:
# Canopy Intercepted water Store (m2/m2)
basin_daily_output_f['canopy_store'].plot(figsize=(17,5))

In [ ]:
# Percent Snow Cover (m2/m2)
basin_daily_output_f['%snow_cover'].plot(figsize=(17,5))

In [ ]:
# Snow Sublimation
basin_daily_output_f['snow_subl'].plot(figsize=(17,5))

In [ ]:
# Spatial variation in transpiration
basin_daily_output_f['trans_var'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['acc_trans'].plot(figsize=(17,5))

In [ ]:
# Potential Evapotranspiration
basin_daily_output_f['pet'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['dC13'].plot(figsize=(17,5))

In [ ]:
# Precipitation (mm)
basin_daily_output_f['precip'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['pcp_assim'].plot(figsize=(17,5))

In [ ]:
# Fraction of Basin that have tree mortality
basin_daily_output_f['mortf'].plot(figsize=(17,5))

In [ ]:
# Maximum Temperature (°C)
basin_daily_output_f['tmax'].plot(figsize=(17,5))

In [ ]:
# Minimum Temperature (°C)
basin_daily_output_f['tmin'].plot(figsize=(17,5))

In [ ]:
# Average Temperature (°C)
basin_daily_output_f['tavg'].plot(figsize=(17,5))

In [ ]:
# Vapor Pressure Deficit (°C)
basin_daily_output_f['vpd'].plot(figsize=(17,5))

In [ ]:
# Snowfall
basin_daily_output_f['snowfall'].plot(figsize=(17,5))

In [ ]:
# _Recharge of water to soil
basin_daily_output_f['recharge'].plot(figsize=(17,5))

In [ ]:
# _Gross Photosynthesis (kgC/m2)
basin_daily_output_f['gpsn'].plot(figsize=(17,5))

In [ ]:
# _ Respiration_ (kgC/m2)
basin_daily_output_f['resp'].plot(figsize=(17,5))

In [ ]:
# Canopy Conductance (mm/s?)
basin_daily_output_f['gs'].plot(figsize=(17,5))

In [ ]:
# Rooting depth
basin_daily_output_f['rootdepth'].plot(figsize=(17,5))

In [ ]:
# Plant Carbon (kgC/m2)
basin_daily_output_f['plantc'].plot(figsize=(17,5))

In [ ]:
# Snow Melt
basin_daily_output_f['snowmelt'].plot(figsize=(17,5))

In [ ]:
# Canopy Sublimation
basin_daily_output_f['canopysubl'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['routedstreamflow'].plot(figsize=(17,5))

In [ ]:
# Snow Intercepted on Canopy
basin_daily_output_f['canopy_snow'].plot(figsize=(17,5))

In [ ]:
# Canopy height
basin_daily_output_f['height'].plot(figsize=(17,5))

In [ ]:
# Canopy Evaporation?
basin_daily_output_f['evap_can'].plot(figsize=(17,5))

In [ ]:
# Litter Evaporation_
basin_daily_output_f['evap_lit'].plot(figsize=(17,5))

In [ ]:
# Soil Evaporation_
basin_daily_output_f['evap_soil'].plot(figsize=(17,5))

In [ ]:
# Litter Carbon_
basin_daily_output_f['litrc'].plot(figsize=(17,5))

In [ ]:
# Downward (from atmosphere) Direct Shortwave Radiation_
basin_daily_output_f['Kdown'].plot(figsize=(17,5))

In [ ]:
# Downward (from atmosphere) Longwave Radiation_
basin_daily_output_f['Ldown'].plot(figsize=(17,5))

In [ ]:
# Reflected (upward) Shortwave Radiation_
basin_daily_output_f['Kup'].plot(figsize=(17,5))

In [ ]:
# Reflected (upward) Longwave Radiation_
basin_daily_output_f['Lup'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave by canopy
basin_daily_output_f['Kstar_can'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave by soil
basin_daily_output_f['Kstar_soil'].plot(figsize=(17,5))

In [ ]:
# Absorbed shortwave bysnow
basin_daily_output_f['Kstar_snow'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by canopy
basin_daily_output_f['Lstar_can'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by soil
basin_daily_output_f['Lstar_soil'].plot(figsize=(17,5))

In [ ]:
# Absorbed longwave by snow
basin_daily_output_f['Lstar_snow'].plot(figsize=(17,5))

In [ ]:
# Latent heat evaporated by canopy
basin_daily_output_f['LE_canopy'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['LE_soil'].plot(figsize=(17,5))

In [ ]:
# LE_snow
basin_daily_output_f['LE_snow'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['Lstar_strat'].plot(figsize=(17,5))

In [ ]:
# 
basin_daily_output_f['canopydrip'].plot(figsize=(17,5))

In [ ]:
# Aerodynamic Conductance
basin_daily_output_f['ga'].plot(figsize=(17,5))

In [ ]:
plt.figure(1)
plt.subplot(2,3,1)

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['precip'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['pet'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_can'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_lit'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['evap_soil'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['trans'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['psn'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['lai'])
plt.legend()

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['precip'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['streamflow'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['return'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['baseflow'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['unsat_drain'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['rz_drainage'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['gw.Qout'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['%sat_area'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['tavg'])
plt.legend()

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax = fig.add_subplot(331)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['sat_def_z'])
plt.legend()
ax = fig.add_subplot(332)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['sat_def'])
plt.legend()
ax = fig.add_subplot(333)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['cap'])
plt.legend()
ax = fig.add_subplot(334)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['gw.storage'])
plt.legend()
ax = fig.add_subplot(335)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['detention_store'])
plt.legend()
ax = fig.add_subplot(336)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['litter_store'])
plt.legend()
ax = fig.add_subplot(337)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['canopy_store'])
plt.legend()
ax = fig.add_subplot(338)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['vpd'])
plt.legend()
ax = fig.add_subplot(339)
plt.plot(basin_daily_output_f.index,basin_daily_output_f['recharge'])
plt.legend()

## 6) Validation between the observation and simulation data.

In [ ]:
path = "obs/"
obs_streamflow = pd.read_csv(path + "Qobs_18_r.csv") #, header=3
obs_streamflow.head()

In [ ]:
start_date = "1936-11-01"
date_index1 = pd.date_range(start_date, periods=len(obs_streamflow), freq='1D')
obs_streamflow_date = obs_streamflow.insert(loc=0, column='Date', value=date_index1.values)
obs_streamflow_date_index = obs_streamflow.set_index('Date')
obs_streamflow_date_index.head()

In [ ]:
obs_streamflow_filt = obs_streamflow_date_index.loc[plt_start_date:plt_end_date]

In [ ]:
# create the plot figure 
plt.figure(figsize=(15,5))
# get the current axis of the plot
ax = plt.gca()
# plot and set label, marker, and markersize
ax.plot(obs_streamflow_filt['discharge (mm)'], label='Observation(mm)', marker="^", markersize=3)
ax.plot(basin_daily_output_f['streamflow'], label='Model Output(mm)', marker="*", markersize=3)
ax.grid(True)
# set the y-axis labels
ax.set_ylabel('Streaflow(m)', fontsize=15)
# setting legend, xticks and yticks fontsizes
plt.legend(fontsize=12)
plt.xticks(rotation=90, fontsize=12)
plt.yticks(fontsize=12)
plt.show()

#### Application of validation method

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt
from pysumma.Validation import validation

In [ ]:
# defind simulation & observation data
lumped_simulation_streamflow = basin_daily_output_f['streamflow'].fillna(0)
observation_streamflow = obs_streamflow_filt['discharge (mm)'].fillna(0)

In [ ]:
# analyze validtation between 1d richards' runoff simulation and observation data.
validation.analysis(observation_streamflow, lumped_simulation_streamflow)

In [ ]:
r2_score(observation_streamflow, lumped_simulation_streamflow)

In [ ]:
#bias NSE log.Q
#monthly, daily, weekly